# RMS Configuration Generator

## 1. Imports & Data Models
Start here by loading the necessary libraries and defining the core data structures (`Resource`, `Cell`, `RMSConfiguration`).

In [26]:
import json
import random
import tkinter as tk
from tkinter import filedialog
import copy
import os
import itertools
import statistics
import math
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Literal
from collections import Counter
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

@dataclass
class Resource:
    resID: int
    cellID: int
    processVector: List[int]
    setupVector: List[int] = field(default_factory=list)

@dataclass
class Cell:
    resources: List[Resource]
    id: int

@dataclass
class RMSConfiguration:
    name: str
    description: str
    cells: List[Cell]

    def to_dict(self) -> Dict:
        return {
            "description": self.description,
            "name": self.name,
            "cells": [
                {
                    "resources": [
                        {
                            "resID": r.resID,
                            "cellID": r.cellID,
                            "processVector": r.processVector,
                            "setupVector": r.setupVector
                        } for r in c.resources
                    ]
                } for c in self.cells
            ]
        }

    @classmethod
    def from_dict(cls, data: Dict) -> 'RMSConfiguration':
        cells = []
        raw_cells = data.get('cells', [])
        current_cell_id = 1
        for c_data in raw_cells:
            resources = []
            for r_data in c_data.get('resources', []):
                res = Resource(
                    resID=r_data['resID'],
                    cellID=r_data.get('cellID', current_cell_id),
                    processVector=r_data['processVector'],
                    setupVector=r_data.get('setupVector', [])
                )
                resources.append(res)
            cells.append(Cell(resources=resources, id=current_cell_id))
            current_cell_id += 1
        
        return cls(
            name=data.get('name', 'New Configuration'),
            description=data.get('description', ''),
            cells=cells
        )


## 2. Analytical Indicators & Helper Functions
Logic for calculating Symmetry, Entropy, Gap, and Smart indicators.
Includes vector generation and case classification helpers.

In [27]:
def calculate_stats(resources: List[Resource]) -> Dict[str, str]:
    if not resources: return {'pt': '0 / 0', 'st': '0 / 0'}
    
    pt_vecs = [p for r in resources for p in r.processVector]
    st_vecs = [s for r in resources for s in r.setupVector]
    
    def get_fmt(vals):
        if not vals: return "0 / 0"
        mean = statistics.mean(vals)
        if len(vals) > 1: stdev = statistics.stdev(vals)
        else: stdev = 0
        return f"{mean:.1f} (σ={stdev:.1f})"
        
    return {'pt': get_fmt(pt_vecs), 'st': get_fmt(st_vecs)}

def generate_vector_generic(nb_families: int, rule_type: str, val_min: int, val_max: int, shared_val: Optional[int] = None, shared_vector: Optional[List[int]] = None) -> List[int]:
    if rule_type == 'Identical':
        val = shared_val if shared_val is not None else 1
        return [val] * nb_families
    elif rule_type == 'Heterogeneous + Resource Dependent':
        return [random.randint(val_min, val_max)] * nb_families
    elif rule_type == 'Identical + Family Dependent':
        if shared_vector: return list(shared_vector)
        return [val_min] * nb_families
    elif rule_type == 'Heterogeneous + Family Dependent':
        return [random.randint(val_min, val_max) for _ in range(nb_families)]
    return []

def get_case_code(res: Resource, cell_resources: List[Resource], vector_attr: str) -> str:
    vec = getattr(res, vector_attr)
    if not vec: return "?"
    is_constant_per_fam = (len(set(vec)) == 1)
    is_identical_across_cell = True
    for r in cell_resources:
        r_vec = getattr(r, vector_attr)
        if r_vec != vec:
            is_identical_across_cell = False; break
    if is_constant_per_fam: return "I" if is_identical_across_cell else "HR"
    else: return "IF" if is_identical_across_cell else "HF"


## 3. Visualization & Reporting
Functions to render the HTML visual representation of cells and configs.

In [28]:
def render_cell_visual(cell: Cell, cw_dict: Optional[Dict], use_simulated=False):
    resources_to_render = []
    mag = 'Zero'
    if cw_dict and use_simulated:
        try:
            mag = cw_dict['mag'].value
            st_rule = cw_dict['st_type'].value
            pt_rule = cw_dict['pt_type'].value
            nb_fam = 3
            if cell.resources and cell.resources[0].processVector: nb_fam = len(cell.resources[0].processVector)
            
            # Simulation logic (simplified)
            pt_vec_shared = [random.randint(1,10) for _ in range(nb_fam)]
            st_vec_shared = [2 for _ in range(nb_fam)]
            for i in range(len(cell.resources)):
                sim_pt = generate_vector_generic(nb_fam, pt_rule, 1, 10, 5, pt_vec_shared)
                sim_st = []
                if mag == 'Zero': sim_st = [0]*nb_fam
                else: sim_st = generate_vector_generic(nb_fam, st_rule, 1, 5, 2, st_vec_shared)
                resources_to_render.append(Resource(i+1, cell.id, sim_pt, sim_st))
        except: resources_to_render = cell.resources
    else:
        resources_to_render = cell.resources
        if cw_dict and 'mag' in cw_dict: mag = cw_dict['mag'].value
        elif resources_to_render and sum([sum(r.setupVector) for r in resources_to_render]) > 0: mag = '>'

    stats = calculate_stats(resources_to_render)
    if not resources_to_render: return "<div>Empty</div>"
    
    first_res = resources_to_render[0]
    code_pt = get_case_code(first_res, resources_to_render, 'processVector')
    code_st = get_case_code(first_res, resources_to_render, 'setupVector')
    if mag == 'Zero': code_st = "0"
    
    fill_pt = "white"; text_col_pt = "black"
    if code_pt == 'IF': fill_pt = "#e0e0e0"
    elif code_pt == 'HR': fill_pt = "#e1bee7" 
    elif code_pt == 'HF': fill_pt = "#ce93d8"
    fill_st = "white"
    if mag != 'Zero' and (mag == '>' or '>' in mag): fill_st = "#ffcc80"
    elif mag != 'Zero': fill_st = "#a5d6a7"
    
    stats_html = f"<div style='font-size:10px; margin-left:5px;'><b>PT</b>:{stats['pt']}<br><b>ST</b>:{stats['st']}</div>"

    # --- Progress Bars Section (Means) ---
    res_bars_html = ""
    max_val = 0
    
    # Calculate Means and Max for scaling
    res_means = []
    for r in resources_to_render:
        m_pt = statistics.mean(r.processVector) if r.processVector else 0
        m_st = statistics.mean(r.setupVector) if r.setupVector else 0
        res_means.append((m_pt, m_st))
        if (m_pt + m_st) > max_val: max_val = m_pt + m_st
    
    if max_val == 0: max_val = 1
    
    for i, r in enumerate(resources_to_render):
        m_pt, m_st = res_means[i]
        
        # Colors: PT=DarkGray, ST=Green (<=PT) or Orange (>PT)
        col_pt = "#555555"
        col_st = "#4caf50" # Green
        if m_st > m_pt: col_st = "#ff9800" # Orange
        
        pct_pt = (m_pt / max_val) * 100
        pct_st = (m_st / max_val) * 100
        
        total_disp = round(m_pt + m_st, 1)
        if total_disp.is_integer(): total_disp = int(total_disp)
        
        # Resource Bar
        res_bars_html += f'''
        <div style="margin-top:2px; font-size:9px; display:flex; align-items:center;">
             <div style="width:15px; text-align:right; margin-right:3px;">M{r.resID}</div>
             <div style="flex-grow:1; background:#eee; height:8px; border-radius:2px; display:flex; overflow:hidden;">
                 <div style="width:{pct_pt}%; background:{col_pt};" title="Mean PT: {round(m_pt,1)}"></div>
                 <div style="width:{pct_st}%; background:{col_st};" title="Mean ST: {round(m_st,1)}"></div>
             </div>
             <div style="margin-left:3px; font-size:8px; color:#666;">{total_disp}</div>
        </div>
        '''

    html = f'''
    <div style="display: inline-block; border: 1px solid #ccc; padding: 5px; margin: 5px; border-radius: 8px; background: #fafafa; min-width: 140px; vertical-align:top;">
        <div style="font-weight: bold; margin-bottom: 5px; border-bottom:1px solid #eee; padding-bottom:2px;">
            Cell #{cell.id}
        </div>
        <div style="display: flex; gap: 10px; align-items:center;">
            <div style="text-align: center;">
                <svg width="40" height="70">
                    <rect x="5" y="5" width="30" height="30" fill="{fill_pt}" rx="4" stroke="#333" stroke-width="1" />
                    <text x="20" y="24" font-size="10" text-anchor="middle" fill="{text_col_pt}" font-weight="bold">{code_pt}</text>
                    <circle cx="20" cy="55" r="12" fill="{fill_st}" stroke="#333" stroke-width="1" />
                    <text x="20" y="59" font-size="10" text-anchor="middle" fill="black">{code_st}</text>
                    <line x1="20" y1="35" x2="20" y2="43" stroke="#888" stroke-width="2" />
                </svg>
            </div>
            {stats_html}
        </div>
        <div style="margin-top:5px; border-top:1px dashed #ddd; padding-top:2px;">
            {res_bars_html}
        </div>
    </div>
    '''
    return html

def get_legend_html():
    return """
    <div style='margin-top:20px; padding:15px; background:#f9f9f9; border:1px solid #ddd; border-radius:5px;'>
        <h3>Legend</h3>
        <ul>
            <li><b>PT/ST Cases</b>: I (Identical), IF (Family Dep), HR (Resource Dep), HF (Fully Hetero).</li>
            <li><b>Stats</b>: Mean (σ=StdDev). Calculated on all vector values in the cell.</li>
        </ul>
    </div>
    """


## 4. Main Configuration Logic
Generation logic for the base configuration (Import / Create New / Rules Application).

In [29]:
current_config: Optional[RMSConfiguration] = None
cell_widgets = {}

out_debug = widgets.Output()
out_viz = widgets.HTML("<i>Load a config to see visualization</i>")
out_final = widgets.Output()
out_json_details = widgets.Output()

latest_generated_config: Optional[RMSConfiguration] = None
latest_report_html: str = ""

# --- UI Config ---
w_fam = widgets.IntText(value=5, description='Families:')
w_cell = widgets.IntText(value=3, description='Cells:')
w_mach = widgets.Text(value="2", description='Mach/Cell:')
w_pt_min = widgets.IntText(value=1, description='PT Min:')
w_pt_max = widgets.IntText(value=10, description='PT Max:')
w_include_nz = widgets.Checkbox(value=True, description='Include (NZ) Ops')

# NEW: Toggle for Live Preview vs Actual Data
w_live_preview = widgets.Checkbox(value=False, description='Live/Simulated Preview')

# Setup Buttons (Global)
btn_zero_setup = widgets.Button(description="No Setup", button_style='')
btn_light_setup = widgets.Button(description="Light Setup", button_style='success')
btn_heavy_setup = widgets.Button(description="Heavy Setup", button_style='warning')

# Rule Buttons
btn_identical_pt = widgets.Button(description="Identical PT", button_style='')
btn_identical_st = widgets.Button(description="Identical ST", button_style='')

def update_viz(*args):
    if not current_config: return
    # If Live Preview is Checked: Use Widgets (Simulated)
    # If Live Preview is Unchecked: Use None (Actual Data from current_config)
    
    use_simulation = w_live_preview.value
    
    html = '<div style="display: flex; flex-wrap: wrap;">'
    for cell in current_config.cells:
        cw = cell_widgets.get(cell.id)
        if cw and use_simulation: 
            html += render_cell_visual(cell, cw, use_simulated=True)
        else: 
            html += render_cell_visual(cell, None, use_simulated=False)
    html += '</div>'
    out_viz.value = html

w_live_preview.observe(update_viz, 'value')

def set_setup_zero(b):
    if not cell_widgets: return
    for w in cell_widgets.values():
        w['mag'].value = 'Zero'
        w['ratio'].value = 0.0
    w_live_preview.value = True # Auto-enable preview when changing settings
    on_gen(b)

def set_setup_light(b):
    if not cell_widgets: return
    for w in cell_widgets.values():
        opts = w['mag'].options
        # Prefer < PT (NZ), then < PT
        if '< PT (NZ)' in opts: w['mag'].value = '< PT (NZ)'
        elif '< PT' in opts: w['mag'].value = '< PT'
        w['ratio'].value = 0.5
    w_live_preview.value = True
    on_gen(b)

def set_setup_heavy(b):
    if not cell_widgets: return
    for w in cell_widgets.values():
        opts = w['mag'].options
        # Prefer > PT, then >= PT
        if '> PT' in opts: w['mag'].value = '> PT'
        elif '>= PT' in opts: w['mag'].value = '>= PT'
        w['ratio'].value = 1.5
    w_live_preview.value = True
    on_gen(b)

def set_identical_pt(b):
    if not cell_widgets: return
    for w in cell_widgets.values():
        w['pt_type'].value = 'Identical'
    w_live_preview.value = True
    on_gen(b)

def set_identical_st(b):
    if not cell_widgets: return
    for w in cell_widgets.values():
        w['st_type'].value = 'Identical'
    w_live_preview.value = True
    on_gen(b)

# Per-Cell Setup Logic
def set_cell_setup(cid, mode, b=None):
    w = cell_widgets.get(cid)
    if not w: return
    if mode == 'Zero':
        w['mag'].value = 'Zero'
        w['ratio'].value = 0.0
    elif mode == 'Light':
        opts = w['mag'].options
        if '< PT (NZ)' in opts: w['mag'].value = '< PT (NZ)'
        elif '< PT' in opts: w['mag'].value = '< PT'
        w['ratio'].value = 0.5
    elif mode == 'Heavy':
        opts = w['mag'].options
        if '> PT' in opts: w['mag'].value = '> PT'
        elif '>= PT' in opts: w['mag'].value = '>= PT'
        w['ratio'].value = 1.5
    # Trigger viz update (simulated)
    w_live_preview.value = True
    # update_viz() # Managed by observe logic or checkbox change? 
    # Actually setting value triggers observer? Yes.
    # But if value was already True, it won't trigger. Call explicitly if needed?
    # Widgets usually trigger observe only on change.
    update_viz()

btn_zero_setup.on_click(set_setup_zero)
btn_light_setup.on_click(set_setup_light)
btn_heavy_setup.on_click(set_setup_heavy)
btn_identical_pt.on_click(set_identical_pt)
btn_identical_st.on_click(set_identical_st)

def refresh_cell_tabs():
    global cell_widgets
    if not current_config: return
    cell_widgets = {}
    children = []
    mag_opts = ['Zero'] + ['< PT', '<= PT', '< PT (NZ)', '<= PT (NZ)', '= PT', '>= PT', '> PT']
    if not w_include_nz.value: mag_opts = [m for m in mag_opts if '(NZ)' not in m]
    
    rule_opts = ['Identical', 'Identical + Family Dependent', 'Heterogeneous + Resource Dependent', 'Heterogeneous + Family Dependent']
    
    for cell in current_config.cells:
        dd_pt = widgets.Dropdown(options=rule_opts, value='Identical', description='PT Rule:', layout=widgets.Layout(width='300px'))
        dd_mag = widgets.Dropdown(options=mag_opts, value='Zero', description='ST Mag:')
        chk_null = widgets.Checkbox(value=False, description='Allow 0', layout=widgets.Layout(width='100px'))
        dd_st = widgets.Dropdown(options=rule_opts, value='Identical', description='ST Rule:', layout=widgets.Layout(width='300px'))
        fl_rat = widgets.FloatText(value=0.5, description='Ratio:', step=0.1, layout=widgets.Layout(width='150px'))
        
        # When user touches these, turn on Preview?
        def on_change(change):
            if change['name'] == 'value':
                 w_live_preview.value = True # Enable preview if user fiddles
                 update_viz()

        for w in [dd_mag, dd_st, fl_rat, dd_pt, chk_null]: w.observe(on_change, 'value')
        cell_widgets[cell.id] = {'mag': dd_mag, 'st_type': dd_st, 'ratio': fl_rat, 'pt_type': dd_pt, 'allow_nulls': chk_null}
        
        # Per-cell Buttons
        btn_cz = widgets.Button(description="Zero", layout=widgets.Layout(width='60px'))
        btn_cl = widgets.Button(description="Light", layout=widgets.Layout(width='60px'), button_style='success')
        btn_ch = widgets.Button(description="Heavy", layout=widgets.Layout(width='60px'), button_style='warning')
        
        # Bind using lambda with default arg to capture current cell.id
        btn_cz.on_click(lambda b, cid=cell.id: set_cell_setup(cid, 'Zero', b))
        btn_cl.on_click(lambda b, cid=cell.id: set_cell_setup(cid, 'Light', b))
        btn_ch.on_click(lambda b, cid=cell.id: set_cell_setup(cid, 'Heavy', b))
        
        children.append(widgets.VBox([
            widgets.HBox([widgets.Label(f"Cell {cell.id}"), btn_cz, btn_cl, btn_ch]),
            dd_pt, 
            widgets.HBox([dd_mag, chk_null]), 
            fl_rat, 
            dd_st
        ]))
    tab.children = children
    for i,c in enumerate(current_config.cells): tab.set_title(i, f"C{c.id}")
    update_viz()

def on_upload_change(change):
    global current_config
    if not change['new']: return
    try:
        file_info = change['new'][0]
        content = json.loads(file_info['content'].tobytes())
        current_config = RMSConfiguration.from_dict(content)
        w_live_preview.value = False # Disable preview to show actual data
        refresh_cell_tabs()
        with out_debug: print(f"Loaded {current_config.name}")
    except Exception as e: 
        with out_debug: print(f"Error: {e}")

def create_random_config(b):
    global current_config
    try:
        factors = [int(x.strip()) for x in w_mach.value.split(',')]
        nb_cells = w_cell.value
        if len(factors) == 1: factors *= nb_cells
        cells = []
        res_id = 1; org = ""
        for c_id in range(1, nb_cells+1):
            nb = factors[c_id-1]; org += str(nb)
            res_list = []
            for _ in range(nb):
                pt = [random.randint(w_pt_min.value, w_pt_max.value) for _ in range(w_fam.value)]
                res_list.append(Resource(res_id, c_id, pt, [0]*w_fam.value))
                res_id += 1
            cells.append(Cell(res_list, c_id))
        name = f"F_{w_fam.value}F{nb_cells}C{res_id-1}M_{org}_1"
        current_config = RMSConfiguration(name, "Random", cells)
        w_live_preview.value = False # Show the generated (actual) config
        refresh_cell_tabs()
    except: return

def randomize_rules(b):
    if not cell_widgets: return
    rule_opts = ['Identical', 'Identical + Family Dependent', 'Heterogeneous + Resource Dependent', 'Heterogeneous + Family Dependent']
    mag_opts = ['Zero'] + ['< PT', '<= PT', '< PT (NZ)', '<= PT (NZ)', '= PT', '>= PT', '> PT']
    for cid, w in cell_widgets.items():
        w['pt_type'].value = random.choice(rule_opts)
        w['st_type'].value = random.choice(rule_opts)
        w['mag'].value = random.choice(mag_opts)
        w['ratio'].value = round(random.uniform(0.1, 2.0), 1)
    w_live_preview.value = True
    on_gen(b)

def make_chaos(b):
    # Standard Chaotic: Unique profile per cell (random combination)
    if not cell_widgets: return
    
    opts_pt = ['Heterogeneous + Family Dependent', 'Heterogeneous + Resource Dependent', 'Identical + Family Dependent']
    opts_st = ['Heterogeneous + Family Dependent', 'Identical + Family Dependent', 'Heterogeneous + Resource Dependent']
    
    deck = list(itertools.product(opts_pt, opts_st))
    random.shuffle(deck)
    deck_cycle = itertools.cycle(deck)
    
    for cid, w in cell_widgets.items():
        pt_val, st_val = next(deck_cycle)
        w['pt_type'].value = pt_val
        w['st_type'].value = st_val
        
        available_mags = list(w['mag'].options)
        preferred = [m for m in available_mags if '>' in m or '=' in m]
        if not preferred: preferred = available_mags 
        
        w['mag'].value = random.choice(preferred)
        w['ratio'].value = round(random.uniform(1.2, 4.0), 1)
    w_live_preview.value = True
    on_gen(b)

def make_super_chaos(b):
    # Super Chaotic: Unique Rules Globally, ANY Magnitude
    if not cell_widgets: return
    n_cells = len(cell_widgets)
    
    all_rules = ['Identical', 'Identical + Family Dependent', 'Heterogeneous + Resource Dependent', 'Heterogeneous + Family Dependent']
    if n_cells > len(all_rules):
        with out_debug: 
            print(f"Warning: You have {n_cells} cells but only {len(all_rules)} rule types. Duplicates will occur.")
    
    # Shuffle for PT and ST separately to maximize variance
    pt_deck = all_rules.copy(); random.shuffle(pt_deck)
    st_deck = all_rules.copy(); random.shuffle(st_deck)
    
    pt_cycle = itertools.cycle(pt_deck)
    st_cycle = itertools.cycle(st_deck)
    
    for cid, w in cell_widgets.items():
        w['pt_type'].value = next(pt_cycle)
        w['st_type'].value = next(st_cycle)
        
        # Allow ANY magnitude
        available_mags = list(w['mag'].options)
        w['mag'].value = random.choice(available_mags)
        w['ratio'].value = round(random.uniform(0.1, 4.0), 1)
    w_live_preview.value = True
    on_gen(b)

def on_gen(b):
    global latest_generated_config, latest_report_html
    if not current_config: return
    new_conf = copy.deepcopy(current_config)
    
    for cell in new_conf.cells:
        cw = cell_widgets.get(cell.id)
        if not cw: continue
        pt_rule = cw['pt_type'].value
        st_rule = cw['st_type'].value
        mag = cw['mag'].value
        rat = cw['ratio'].value
        allow_nulls = cw['allow_nulls'].value
        nb_fam = 3
        if cell.resources and cell.resources[0].processVector: nb_fam = len(cell.resources[0].processVector)
        
        # Always regenerate PT now
        pt_shared = random.randint(w_pt_min.value, w_pt_max.value)
        pt_vec_shared = [random.randint(w_pt_min.value, w_pt_max.value) for _ in range(nb_fam)]
        # Enforce non-flat if Family Dependent to avoid visual collision with Identical
        if 'Family Dependent' in pt_rule and nb_fam > 1 and w_pt_min.value != w_pt_max.value:
            attempts = 0
            while len(set(pt_vec_shared)) == 1 and attempts < 10:
                pt_vec_shared = [random.randint(w_pt_min.value, w_pt_max.value) for _ in range(nb_fam)]
                attempts += 1

        for res in cell.resources:
             res.processVector = generate_vector_generic(nb_fam, pt_rule, w_pt_min.value, w_pt_max.value, pt_shared, pt_vec_shared)
        
        all_pt_mean = 1
        pts = [p for r in cell.resources for p in r.processVector]
        if pts: all_pt_mean = sum(pts)/len(pts)
        st_target = 0
        core_mag = mag.replace(" (NZ)", "").strip()
        if core_mag == '= PT': st_target = all_pt_mean
        elif '<' in core_mag: st_target = all_pt_mean * rat
        elif '>' in core_mag: st_target = all_pt_mean * rat
        st_target = max(0, int(st_target))
        if 'NZ' in mag: st_target = max(1, st_target)
        st_min = 0; st_max = 0
        if mag != 'Zero':
            low = 1 if ('NZ' in mag or not allow_nulls) else 0
            st_min = max(low, int(st_target*0.8))
            st_max = max(low, int(st_target*1.2))
        st_shared_val = st_target
        st_shared_vec = [random.randint(st_min, st_max) for _ in range(nb_fam)]
        
        # Enforce non-flat if Family Dependent (ST)
        if 'Family Dependent' in st_rule and nb_fam > 1 and st_min != st_max:
            attempts = 0
            while len(set(st_shared_vec)) == 1 and attempts < 10:
                st_shared_vec = [random.randint(st_min, st_max) for _ in range(nb_fam)]
                attempts += 1

        for res in cell.resources:
             if mag == 'Zero': res.setupVector = [0]*nb_fam
             else: res.setupVector = generate_vector_generic(nb_fam, st_rule, st_min, st_max, st_shared_val, st_shared_vec)
    
    latest_generated_config = new_conf
    # Render full report
    report_html = '<div style="display: flex; flex-wrap: wrap;">'
    for cell in new_conf.cells: report_html += render_cell_visual(cell, cell_widgets.get(cell.id), use_simulated=False)
    report_html += '</div>'
    latest_report_html = report_html
    
    with out_final:
        clear_output()
        print(f"Generated: {new_conf.name}")
        display(HTML(report_html))
    
    with out_json_details:
        clear_output(wait=True)
        print(json.dumps(new_conf.to_dict(), indent=2))

def on_save(b):
    if not latest_generated_config: return
    try:
        # Logic for saving
        base = latest_generated_config.name
        folder = f"{len(latest_generated_config.cells)} Cells"
        # Auto-inc
        v = 1; pfx = base.rsplit('_',1)[0] if base[-1].isdigit() else base
        while os.path.exists(os.path.join(folder, f"{pfx}_{v}")): v+=1
        name = f"{pfx}_{v}"
        path = os.path.join(folder, name)
        os.makedirs(path, exist_ok=True)
        with open(os.path.join(path, f"{name}.json"), 'w') as f: f.write(json.dumps(latest_generated_config.to_dict(), indent=2))
        with open(os.path.join(path, f"{name}_report.html"), 'w') as f: f.write(f"<html><body>{latest_report_html}{get_legend_html()}</body></html>")
        with out_final: print(f"Saved to {path}")
    except Exception as e:
        with out_final: print(f"Error saving: {e}") 

btn_create = widgets.Button(description="Create New"); btn_create.on_click(create_random_config)
btn_rand_rules = widgets.Button(description="Randomize Rules"); btn_rand_rules.on_click(randomize_rules)
btn_rand_rules.style.button_color = '#666666'; btn_rand_rules.style.text_color = 'white'

# Chaotic Buttons
btn_chaotic = widgets.Button(description="Chaotic", tooltip="Unique cell profiles per cell")
btn_chaotic.style.button_color = 'black'; btn_chaotic.style.text_color = 'white'
btn_chaotic.on_click(make_chaos)

# Updated Super Chaotic Button (Red, Global Rules, Any Mag)
btn_super_chaotic = widgets.Button(description="Super Chaotic", button_style='danger', tooltip="PT/ST Rules do not repeat across cells")
btn_super_chaotic.on_click(make_super_chaos)

uploader = widgets.FileUpload(accept='.json', multiple=False); uploader.observe(on_upload_change, names='value')
tab_init = widgets.Tab([widgets.VBox([widgets.Label("Create"), w_fam, w_cell, w_mach, w_pt_min, w_pt_max, w_include_nz, btn_create]), widgets.VBox([widgets.Label("Import"), uploader])])
tab_init.set_title(0,'Create'); tab_init.set_title(1,'Import')
tab = widgets.Tab()
# Revised config area with buttons
config_area = widgets.VBox([
    widgets.HBox([widgets.Label("Presets:"), btn_zero_setup, btn_light_setup, btn_heavy_setup]),
    widgets.HBox([widgets.Label("Rules:"), btn_identical_pt, btn_identical_st]),
    widgets.HBox([widgets.Label("Random:"), btn_rand_rules, btn_chaotic, btn_super_chaotic]),
    w_live_preview,
    tab
])
btn_gen = widgets.Button(description="(Re)generate", button_style='success'); btn_gen.on_click(on_gen)
btn_save = widgets.Button(description="Save", button_style='primary'); btn_save.on_click(on_save)

acc_json = widgets.Accordion(children=[out_json_details])
acc_json.set_title(0, 'View Configuration JSON')

display(widgets.VBox([widgets.HTML("<h4>Configuration</h4>"), tab_init, widgets.HTML("<hr>"), out_viz, widgets.HTML("<hr>"), config_area, widgets.HBox([btn_gen, btn_save]), out_debug, out_final, acc_json]))



## 5. Expansion & Variants Block
Module to generate system variants by adding X resources to cells combinatorially.

In [30]:

w_expand_x_str = widgets.Text(value="1", description='Add X Res:', placeholder="e.g. 1 or 1,2,1")
w_allow_dupe = widgets.Checkbox(value=True, description='Allow Duplicates (e.g. AA)')
btn_est_var = widgets.Button(description="Show All Variants (Preview)", button_style='info')
btn_gen_mem = widgets.Button(description="Generate (Memory)", button_style='warning')
btn_save_disk = widgets.Button(description="Save to Disk", button_style='success', disabled=False)
# Safe Save Features
btn_overwrite = widgets.Button(description="Overwrite", button_style='danger', layout=widgets.Layout(display='none'))
btn_increment = widgets.Button(description="Save as New (Incr)", button_style='primary', layout=widgets.Layout(display='none'))

out_var = widgets.Output() # Kept for messages/logs
# Textarea for copy-friendly list
w_preview_area = widgets.Textarea(
    value="",
    placeholder="Variants will appear here...",
    description='Preview:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='300px')
)

candidate_variants = [] # Tuples of (RMSConfiguration, RelativePath)
candidate_details = [] # Strings describing the variant
current_save_target = ""

def get_combinations_for_cell_profiles(cell, k, allow_dupes=True):
    # Returns (actual_combinations, theoretical_count)
    # actual_combinations is list of Dictionaries: [ {'ids': (1,2), 'profs': (P1, P2)}, ... ]
    
    ids = [r.resID for r in cell.resources]
    # Map ID -> Profile
    id_to_prof = {r.resID: (tuple(r.processVector), tuple(r.setupVector)) for r in cell.resources}
    profiles = [id_to_prof[i] for i in ids]
    
    # 1. Calculate Theoretical (ID-based)
    if allow_dupes:
        nb_theoretical = len(list(itertools.combinations_with_replacement(ids, k)))
    else:
        nb_theoretical = len(list(itertools.combinations(ids, k)))
        
    actual_combs = []
    
    # 2. Calculate Actual
    if allow_dupes:
        # ID-Based Expansion (Slot-based)
        # We explicitly generate combinations of IDs.
        id_combs = list(itertools.combinations_with_replacement(ids, k))
        for id_tuple in id_combs:
            # We map back to profiles for the functional part, but keep IDs for description/uniqueness
            prof_tuple = tuple(id_to_prof[i] for i in id_tuple)
            actual_combs.append({'ids': id_tuple, 'profs': prof_tuple})
    else:
        # Profile-Based Smart De-duplication
        # We work on unique profiles.
        
        # FIX: Sort unique profiles by the ID of the first resource that has this profile.
        # This ensures that if M1 has Profile A and M2 has Profile B, Profile A is processed first.
        
        prof_to_min_id = {}
        for r_id, prof in id_to_prof.items():
            if prof not in prof_to_min_id: prof_to_min_id[prof] = r_id
            else: prof_to_min_id[prof] = min(prof_to_min_id[prof], r_id)
            
        unique_profs = sorted(list(set(profiles)), key=lambda p: prof_to_min_id[p])
        
        prof_combs = list(itertools.combinations_with_replacement(unique_profs, k))
        for prof_tuple in prof_combs:
            actual_combs.append({'ids': None, 'profs': prof_tuple})
            
    # Sort for deterministic output
    # Sort by IDs if present, else by Profiles (which are tuples of tuples)
    actual_combs.sort(key=lambda x: x['ids'] if x['ids'] else x['profs'])
    return actual_combs, nb_theoretical

def resolve_profile_names_and_ids(cell, item_dict, allow_reuse=True):
    # If we have specific IDs, use them.
    if item_dict['ids']:
        return [(item_dict['profs'][i], item_dict['ids'][i]) for i in range(len(item_dict['ids']))]
        
    # Otherwise fallback to profile mapping
    profiles_tuple = item_dict['profs']
    
    # Construct inventory map
    inv_map = {}
    for r in cell.resources:
        p = (tuple(r.processVector), tuple(r.setupVector))
        if p not in inv_map: inv_map[p] = []
        inv_map[p].append(r.resID)
    for k in inv_map: inv_map[k].sort()
    
    resolved = []
    current_map = copy.deepcopy(inv_map)
    
    for p in profiles_tuple:
        candidates = current_map.get(p, [])
        if not candidates: 
            resolved.append((p, -1))
            continue
        picked_id = candidates[0]
        resolved.append((p, picked_id))
    return resolved

def parse_depths(nb_cells, depth_str):
    try:
        parts = [int(p.strip()) for p in depth_str.split(',')]
        if len(parts) == 1: return parts * nb_cells
        if len(parts) != nb_cells: return None
        return parts
    except: return None

def vector_sort_key(vec):
    nz_count = sum(1 for x in vec if x > 0)
    idx_if_pure = -1
    if nz_count == 1:
        idx_if_pure = [i for i,x in enumerate(vec) if x > 0][0]
    return (nz_count, idx_if_pure, vec)

def on_est_variants(b):
    w_preview_area.value = "Calculating..."
    if not latest_generated_config:
        w_preview_area.value = "No base config!"
        return
    parts = parse_depths(len(latest_generated_config.cells), w_expand_x_str.value)
    if not parts:
        w_preview_area.value = "Invalid Depths"
        return

    allow = w_allow_dupe.value
    base = latest_generated_config
    
    levels = []
    is_specific = (',' in w_expand_x_str.value) and (len(parts) == len(base.cells))
    if is_specific: levels.append( ("Specific", [parts]) )
    else:
        max_k = parts[0]
        for k in range(1, max_k+1):
            valid_vecs = [v for v in itertools.product(range(k+1), repeat=len(base.cells)) if sum(v) == k]
            valid_vecs.sort(key=vector_sort_key)
            levels.append( (f"Level_{k}", valid_vecs) )

    org_counters = Counter()
    total_theo = 0
    total_actual = 0
    
    buffer = []
    buffer.append(f"--- Exhaustive Variant List ---")
    if allow: buffer.append("(Mode: Slot-Based / Duplicates Allowed)")
    else: buffer.append("(Mode: Profile-Based / Smart De-duplication / Sorted by ID)")
    buffer.append("(Future Name) : Additions")
    
    for lbl, vecs in levels:
        for v_dist in vecs:
            # Calculate options for this vector
            cell_combs = []
            vec_theo = 1
            
            for idx, c in enumerate(base.cells):
                c_combs, c_theo = get_combinations_for_cell_profiles(c, v_dist[idx], allow)
                cell_combs.append(c_combs)
                vec_theo *= c_theo
            
            total_theo += vec_theo
            
            # Expand actual
            iterator = itertools.product(*cell_combs)
            for v_dicts in iterator:
                total_actual += 1
                
                temp_counts = [len(c.resources) for c in base.cells]
                desc_parts = []
                
                for c_idx, item_dict in enumerate(v_dicts):
                        if not item_dict['profs']: continue
                        resolved = resolve_profile_names_and_ids(base.cells[c_idx], item_dict, allow)
                        additions = []
                        for (p, oid) in resolved:
                            additions.append(f"M{oid}")
                            temp_counts[c_idx] += 1
                        desc_parts.append(f"C{c_idx+1}: {', '.join(additions)}")

                org_str = "".join([str(x) for x in temp_counts])
                org_counters[org_str] += 1
                
                nb_fam = 5
                if base.cells and base.cells[0].resources: nb_fam = len(base.cells[0].resources[0].processVector)
                
                futur_name = f"F_{nb_fam}F{len(base.cells)}C{sum(temp_counts)}M_{org_str}_{org_counters[org_str]}"
                buffer.append(f"({futur_name}) : {' | '.join(desc_parts)}")
    
    diff = total_theo - total_actual
    buffer.append("-" * 40)
    buffer.append(f"Total Unique Variants: {total_actual}")
    if not allow:
            buffer.append(f"Identical/Redundant Variants Collapsed: {diff}")
            
    w_preview_area.value = "\n".join(buffer)

def on_gen_mem(b):
    global candidate_variants, candidate_details
    candidate_variants = []
    candidate_details = []
    btn_overwrite.layout.display = 'none'
    btn_increment.layout.display = 'none'
    btn_save_disk.disabled = True
    
    if not latest_generated_config: return
    parts = parse_depths(len(latest_generated_config.cells), w_expand_x_str.value)
    if not parts: return
    allow = w_allow_dupe.value
    base = latest_generated_config
    
    with out_var: 
        clear_output()
        print("Generating in memory...")
    
    levels = []
    is_specific = (',' in w_expand_x_str.value) and (len(parts) == len(base.cells))
    if is_specific: levels.append( ("Specific", [parts]) )
    else:
        max_k = parts[0]
        for k in range(1, max_k+1):
            valid_vecs = [v for v in itertools.product(range(k+1), repeat=len(base.cells)) if sum(v) == k]
            valid_vecs.sort(key=vector_sort_key)
            levels.append( (f"Level_{k}", valid_vecs) )

    org_counters = Counter()
    total_theo = 0
    total_actual = 0
    
    for lvl_name, vecs in levels:
        for v_dist in vecs:
            cell_combs = []
            vec_theo = 1
            for idx, c in enumerate(base.cells):
                c_combs, c_theo = get_combinations_for_cell_profiles(c, v_dist[idx], allow)
                cell_combs.append(c_combs)
                vec_theo *= c_theo
            total_theo += vec_theo
            
            variants = list(itertools.product(*cell_combs))
            for v_dicts in variants:
                total_actual += 1
                new_c = copy.deepcopy(base)
                desc_parts = []
                
                for c_i, item_dict in enumerate(v_dicts):
                    if not item_dict['profs']: continue
                    resolved = resolve_profile_names_and_ids(base.cells[c_i], item_dict, allow)
                    cell = new_c.cells[c_i]
                    rid = max([r.resID for r in cell.resources])
                    additions = []
                    
                    for (p, oid) in resolved:
                        rid+=1
                        pt_vec, st_vec = p
                        cell.resources.append(Resource(rid, cell.id, list(pt_vec), list(st_vec)))
                        additions.append(f"M{oid}")
                        
                    desc_parts.append(f"C{c_i+1}: {','.join(additions)}")
                
                # Name
                org_parts = [str(len(c.resources)) for c in new_c.cells]
                org_str = "".join(org_parts)
                total_res = sum(len(c.resources) for c in new_c.cells)
                nb_fam = 5
                if new_c.cells and new_c.cells[0].resources: nb_fam = len(new_c.cells[0].resources[0].processVector)
                
                org_counters[org_str] += 1
                new_name = f"F_{nb_fam}F{len(new_c.cells)}C{total_res}M_{org_str}_{org_counters[org_str]}"
                new_c.name = new_name
                
                # For basic logic, path is simpler
                rel_path = os.path.join(lvl_name, f"Var_{len(candidate_variants)+1}")
                candidate_variants.append((new_c, rel_path))
                candidate_details.append(f"({new_name}) : {' | '.join(desc_parts)}")

    btn_save_disk.disabled = False
    diff = total_theo - total_actual
    with out_var: 
        clear_output()
        print(f"Generated {len(candidate_variants)} variants (Sorted, Unique).")
        if not allow: print(f"Collapsed {diff} identical variants.")
        print("(Name) : Description")
        for detail in candidate_details:
             print(f"  {detail}")
        print("\nReady to Save.")

def on_save_variants(b):
    # --- Generation Step ---
    global candidate_variants, candidate_details
    candidate_variants = []
    candidate_details = []
    
    if not latest_generated_config: return
    parts = parse_depths(len(latest_generated_config.cells), w_expand_x_str.value)
    if not parts: 
        with out_var: print("Invalid depths.")
        return
        
    allow = w_allow_dupe.value
    base = latest_generated_config
    
    with out_var: 
        clear_output()
        print("Generating and saving...")
    
    levels = []
    is_specific = (',' in w_expand_x_str.value) and (len(parts) == len(base.cells))
    if is_specific: levels.append( ("Specific", [parts]) )
    else:
        max_k = parts[0]
        for k in range(1, max_k+1):
            valid_vecs = [v for v in itertools.product(range(k+1), repeat=len(base.cells)) if sum(v) == k]
            valid_vecs.sort(key=vector_sort_key)
            levels.append( (f"Level_{k}", valid_vecs) )

    org_counters = Counter()
    
    # 1. Add Level 0 (Base Config)
    # Use a generic name 'Base' for the path, but the file will keep its original name
    candidate_variants.append((copy.deepcopy(base), "Level_0/Base"))
    candidate_details.append(f"({base.name}) : Base Configuration (No added resources)")

    # 2. Generate Variants
    gen_counter = 0
    
    for lvl_name, vecs in levels:
        for v_dist in vecs:
            cell_combs = []
            for idx, c in enumerate(base.cells):
                c_combs, c_theo = get_combinations_for_cell_profiles(c, v_dist[idx], allow)
                cell_combs.append(c_combs)

            variants = list(itertools.product(*cell_combs))
            for v_dicts in variants:
                new_c = copy.deepcopy(base)
                
                for c_i, item_dict in enumerate(v_dicts):
                    if not item_dict['profs']: continue
                    resolved = resolve_profile_names_and_ids(base.cells[c_i], item_dict, allow)
                    cell = new_c.cells[c_i]
                    rid = max([r.resID for r in cell.resources])
                    
                    for (p, oid) in resolved:
                        rid+=1
                        pt_vec, st_vec = p
                        cell.resources.append(Resource(rid, cell.id, list(pt_vec), list(st_vec)))
                
                # Name
                org_parts = [str(len(c.resources)) for c in new_c.cells]
                org_str = "".join(org_parts)
                total_res = sum(len(c.resources) for c in new_c.cells)
                nb_fam = 5
                if new_c.cells and new_c.cells[0].resources: nb_fam = len(new_c.cells[0].resources[0].processVector)
                
                org_counters[org_str] += 1
                new_name = f"F_{nb_fam}F{len(new_c.cells)}C{total_res}M_{org_str}_{org_counters[org_str]}"
                new_c.name = new_name
                
                gen_counter += 1
                rel_path = os.path.join(lvl_name, f"Var_{gen_counter}")
                candidate_variants.append((new_c, rel_path))

    # --- Save Step ---
    if not candidate_variants: 
        with out_var: print("No variants generated.")
        return
    
    # Open directory selector
    root = tk.Tk()
    root.withdraw() 
    root.wm_attributes('-topmost', 1)
    selected_dir = filedialog.askdirectory(title="Select Destination Folder")
    root.destroy()
    
    if not selected_dir:
        with out_var: print("Save cancelled.")
        return

    try:
        saved_count = 0
        base_name = latest_generated_config.name if latest_generated_config else "Unknown"
        rms_root = os.path.join(selected_dir, "RMS", base_name)
        all_levels_dir = os.path.join(rms_root, "All Levels")
        os.makedirs(all_levels_dir, exist_ok=True)
        
        for config, rel_path in candidate_variants:
             full_dir = os.path.join(rms_root, os.path.dirname(rel_path))
             os.makedirs(full_dir, exist_ok=True)
             
             fname = config.name
             json_data = json.dumps(config.to_dict(), indent=2)
             
             with open(os.path.join(full_dir, f"{fname}.json"), 'w') as f:
                 f.write(json_data)
             
             report_html = '<div style="display: flex; flex-wrap: wrap;">'
             for cell in config.cells: 
                 # Uses the simulation toggle from global context to determine visual? 
                 # No, for report we usually want actual. But wait...
                 # The user wants "Live Preview" status to generally affect things?
                 # Usually static reports should show the ACTUAL data of the variant.
                 # The variant 'config' object HAS the data. 
                 # So we pass cw=None and use_simulated=False.
                 report_html += render_cell_visual(cell, None, use_simulated=False)
             report_html += '</div>'
             
             full_html_content = f"<html><body><h3>{fname}</h3>{report_html}{get_legend_html()}</body></html>"
             
             with open(os.path.join(full_dir, f"{fname}.html"), 'w') as f:
                 f.write(full_html_content)

             with open(os.path.join(all_levels_dir, f"{fname}.json"), 'w') as f:
                 f.write(json_data)
             
             saved_count += 1
             
        with out_var:
            print(f"\nSuccessfully generated & saved {saved_count} variants (Including Level 0 Base).")
            print(f"Location: {rms_root}")
            
    except Exception as e:
        with out_var: print(f"Error saving: {e}")

# Bindings
btn_est_var.on_click(on_est_variants)
btn_save_disk.on_click(on_save_variants)

# Final Display
display(widgets.VBox([
    widgets.HTML("<h4>Expansion (Variants)</h4>"),
    widgets.HBox([w_expand_x_str, w_allow_dupe]),
    widgets.HBox([btn_est_var, btn_save_disk, btn_overwrite, btn_increment]),
    w_preview_area,
    out_var
]))
